In [1]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.layers.core import Dense, Activation, Dropout
from datetime import datetime
import time
from keras.activations import *
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
scaler  = MinMaxScaler()
df_train =0
df_test  =0
carpan =1
name ="M1"

"""def TimeStamps(dataframe_Date):
    dTime = dataframe_Date
    if name == ('Weekly'or 'Daily' or 'Monthly'):
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d').timestamp() for i in range(len(dTime))]))#%H:%M:%S
    else :
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d %H %M %S').timestamp() for i in range(len(dTime))]))
"""
import dill
try:
    
    df_train = read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'_hazirVeri_train_dongulu.csv', engine='python')
    df_test  = read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'_hazirVeri_test_dongulu.csv', engine='python')
    #df_train['Date'] = TimeStamps(df_train['Date'])
    #df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+name+'_dataframes_dongulu.db')
    """
    dill.load_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+name+'_dataframes_dongulu.db')
    """
except:
    
    df_train = read_csv('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'_Jan.csv', engine='python')
    df_test  = read_csv('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'_Feb.csv' , engine='python')
    #df_train['Date'] = TimeStamps(df_train['Date'])
    #df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+name+'_dataframes_JanFeb_dongulu.db')
    """
    dill.load_session('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+name+'_dataframes_JanFeb_dongulu.db')
    """
df_train,df_test

In [ ]:
sin = df_train['month_sin']
cos = df_train['month_cos']
sin = df_train['weekly_sin'] + sin
cos = df_train['weekly_cos'] + cos
sin = df_train['day_sin'] + sin
cos = df_train['day_cos'] + cos
sin = df_train['hour_sin'] + sin
cos = df_train['hour_cos'] + cos
sin = df_train['minute_sin'] + sin
cos = df_train['minute_cos'] + cos

date_train =  pd.concat([sin,cos],axis=1)

sin = df_test['month_sin']
cos = df_test['month_cos']
sin = df_test['weekly_sin'] + sin
cos = df_test['weekly_cos'] + cos
sin = df_test['day_sin'] + sin
cos = df_test['day_cos'] + cos
sin = df_test['hour_sin'] + sin
cos = df_test['hour_cos'] + cos
sin = df_test['minute_sin'] + sin
cos = df_test['minute_cos'] + cos

date_test =  pd.concat([sin,cos],axis=1)


In [ ]:
model = Sequential()
giris = 2
cikis = 4
def SetModel(units,train,dense,activations='relu',optimizer='adam', loss='mse'):
    print("Set Mode train shape 0 : ",train.shape[0])
    model.add(LSTM(units, activation=activations,input_shape=(train.shape[1],giris)))
    """
    #model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    #model.add(Dense(512, activation=activations))
    model.add(Dense(dense))
    model.compile(optimizer, loss)
    """
    #model.add(Dense(256*4))
    #model.add(Dropout(0.2))
    #model.add(Dense(256*2))
    #model.add(Dropout(0.2))
    #model.add(Dense(256))
    #model.add(Dropout(0.2))
    #model.add(Dense(256))
    #model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Dense(cikis))
    opt = tf.keras.optimizers.SGD(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
    print(model.summary())
    
def setupDataFrame(dataFrame,train_Dataframe):
    #X
    X_Data = dataFrame
    print("Orjinal Shape X :",X_Data.shape)
    X_Data = np.array(X_Data).reshape((len(X_Data),1,giris))
    print("Yeni Shape X :",X_Data.shape)
    #X_Data = scaler.fit_transform(X_Data)#.round(5)

    #Y
    Y_Data = train_Dataframe
    Y_Data =  pd.concat([Y_Data['Open'],Y_Data['High'],Y_Data['Low'],Y_Data['Close']],axis=1)
    Y_Data = (Y_Data*carpan).values#.round(5)
    print("Orjinal Shape Y :",Y_Data.shape)
    Y_Data = np.array(Y_Data).reshape((len(Y_Data),1,cikis))
    #Y_Data = scaler.fit_transform(Y_Data)#.round(5)
    print("Yeni Shape Y :",Y_Data.shape)
    #print(X_t,y_t)
    return X_Data,Y_Data

def FUNC_Time(train_dataframe,test_dataframe):
    #X_train #Y_train
    X_train,y_train =setupDataFrame(date_train,train_dataframe)
    #X_test #y_test
    X_test,y_test = setupDataFrame(date_test,test_dataframe)
    #SetModel
    SetModel(units=2,train=X_train,dense=cikis)
    #print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    print(X_train,"\n",y_train,"\n",X_test,"\n",y_test,"\n")
    return X_train,y_train,X_test,y_test

X_train,y_train,X_test,y_test = FUNC_Time(df_train,df_test)

In [ ]:
#for i in range(100):
history = model.fit(X_train,y_train, epochs=10,batch_size=1024,validation_data=(X_train,y_train))  
predict = model.predict(X_test)#.round(5)
print(predict,y_test)


h1 units 16 1.8535e-06
h1 units 32 1.9282e-06

In [ ]:
predict = model.predict(X_test)#.round(5)

In [ ]:
predict,y_test

In [ ]:
print(predict)
print(((predict-y_test)))#.round(5)
predict = scaler.inverse_transform(predict)
y_test  = scaler.inverse_transform(y_test)

fark2 = predict[:50]-y_test[:50]

predict = pd.DataFrame(((predict)),columns=['High_Predict','Low_Predict','Close_Predict'])
y_test = pd.DataFrame(((y_test)),columns=['High_Reel','Low_Reel','Close_Reel'])
X_test = pd.DataFrame(((X_test)),columns=['Open'])


found  = pd.concat([X_test,predict],axis=1)
expect = pd.concat([X_test,y_test],axis=1)
print((predict[i]/carpan).round(5),(y_test[i]/carpan).round(5))
print(fark2,"\n \n",expect,"\n \n",found)

In [ ]:
test= X_test.reshape(56086,2)
yy   = y_test.reshape(56086,3)
pre = np.array(predict).reshape(56086,3)
plt.plot(test[0],[yy[0],yy[1],yy[2]],color='blue')
plt.plot(test[0],[pre[0],pre[1],pre[2]],color='red')

In [ ]:
yeni = np.array([1.09160])

model.predict(yeni.reshape(-1,1))#.round(5)

In [ ]:
#from tensorflow.python.client import device_lib
#import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
#print(device_lib.list_local_devices())